# CPSC 330 Lecture 6

#### Lecture outline

- Announcements

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import tree 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_blobs
from sklearn.compose import ColumnTransformer

from plot_classifier import plot_classifier

## Announcements

- Thanks to those who have filled out the survey on Canvas! 
  - Please fill out the survey if you haven't already.
- Per student feedback, the midterm on Feb 13 will start 11:05 and run for 75 minutes. 
- hw3 due Sunday at 6pm
- Extra office hour on Friday at 9am (Nam) 
  - This time slot might become permanent, see calendar.
- I have to run off after class - please post questions on Piazza.

## This week's dataset: income prediction from adult census data (5 min)


- We will use a classic dataset from the UCI dataset repository.
  - This one is about predicting income from adult census data.
  - You can download the dataset [here](https://www.kaggle.com/uciml/adult-census-income).
  - To run the code, you will need to download it and put it in the `data` folder.

In [85]:
df = pd.read_csv('data/adult.csv')
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


## EDA and pandas profiling (15 min)

Let's start with some exploratory data analysis (EDA):

In [83]:
df.shape

(32561, 15)

In [86]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

We will split the data even before doing our exploration, so that we don't look at the test data:

In [87]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=123)

In [88]:
df_train.describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,26048.000000,2.604800e+04,26048.000000,26048.000000,26048.000000,26048.000000
mean,38.586686,1.892295e+05,10.070485,1075.695754,87.629991,40.433239
std,13.619181,1.050005e+05,2.572231,7334.297499,404.192112,12.346313
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175830e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.777850e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.368852e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.366120e+06,16.000000,99999.000000,4356.000000,99.000000


Let's explore the target value - this is often useful!

In [89]:
df_train[["income"]].describe()

,income
count,26048
unique,2
top,<=50K
freq,19810


- `pandas_profiling` is a fancy tool for EDA
- The term "profiling" is a bit unfortunate - this has nothing to do with timing code execution.

In [90]:
from pandas_profiling import ProfileReport

This next line can take several minutes...

In [ ]:
profile = ProfileReport(df, title='Pandas Profiling Report') #, minimal=True)

In [ ]:
# profile.to_file('profile_report.html')

In [91]:
profile.to_notebook_iframe()

## Break (5 min)

## Dealing with missing data (15 min)

In [118]:
df_train.sort_index().head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [119]:
df_train_nan = df_train.replace('?',np.NaN)
df_test_nan = df_test.replace('?',np.NaN)

df_train_nan.sort_index().head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [122]:
X_train_nan = df_train_nan.drop(['income'], axis=1)
X_test_nan = df_test_nan.drop(['income'], axis=1)

- If we refer back to the profile report, we see that we only have missing values in the categorical features.

In [123]:
from sklearn.impute import SimpleImputer

In [124]:
imp = SimpleImputer(strategy='most_frequent')

- This imputer is another transformer, like the other ones we've seen.
- The "most_frequent" strategy puts in the most frequent value.
- There are also strategies for numeric variables, like taking the mean or median value.

In [125]:
imp.fit(X_train_nan[categorical_features]);

In [126]:
X_train_imp_cat = pd.DataFrame(imp.transform(X_train_nan[categorical_features]),
                           columns=categorical_features, index=X_train_nan.index)
X_test_imp_cat = pd.DataFrame(imp.transform(X_test_nan[categorical_features]),
                           columns=categorical_features, index=X_test_nan.index)

X_train_imp = X_train_nan.copy()
X_train_imp.update(X_train_imp_cat)

X_test_imp = X_test_nan.copy()
X_test_imp.update(X_test_imp_cat)

We can see the missing values filled in:

In [127]:
X_train_nan.sort_index().head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States


In [128]:
X_train_imp.sort_index().head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,Private,77053,HS-grad,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
2,66,Private,186061,Some-college,10,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States


## Try different classifiers (10 min)

- Let's use cross-validation with our training set.
- This is implemented in `cross_val_score`

In [136]:
from sklearn.model_selection import cross_val_score

Example:

In [137]:
lr = LogisticRegression(max_iter=300)

In [138]:
scores = cross_val_score(lr, X_train_scale_ohe, y_train, cv=5)

In [139]:
scores

array([0.84760077, 0.85028791, 0.8452975 , 0.85813016, 0.85563448])

In [140]:
1.0 - np.mean(scores)

0.14860983629028313

There is also the slightly more sophisticated `cross_validate`, which gives us some extra information:

In [141]:
from sklearn.model_selection import cross_validate

In [142]:
lr = LogisticRegression(max_iter=300)

In [143]:
scores = cross_validate(lr, X_train_scale_ohe, y_train, cv=5, return_train_score=True)

In [144]:
scores

{'fit_time': array([1.58401108, 1.411165  , 0.99453616, 1.17865992, 0.94757175]),
 'score_time': array([0.01569891, 0.01687503, 0.017941  , 0.01524305, 0.01524019]),
 'test_score': array([0.84760077, 0.85028791, 0.8452975 , 0.85813016, 0.85563448]),
 'train_score': array([0.85449659, 0.85435263, 0.85526442, 0.85253611, 0.85157637])}

In [145]:
scores_df = pd.DataFrame(scores).drop(columns=['score_time'])
scores_df

,fit_time,test_score,train_score
0,1.584011,0.847601,0.854497
1,1.411165,0.850288,0.854353
2,0.994536,0.845298,0.855264
3,1.178660,0.858130,0.852536
4,0.947572,0.855634,0.851576


In [146]:
scores_df.mean()

fit_time       1.223189
test_score     0.851390
train_score    0.853645
dtype: float64

Now let's try some different classifiers:

In [ ]:
models = {'decision tree'      : DecisionTreeClassifier(),
          'logistic regression': LogisticRegression(max_iter=300),
          'RBF SVM'            : SVC(max_iter=1500), 
          'random forest'      : RandomForestClassifier(), 
         }
avg_scores = dict()

for model_name, model in models.items():
    print(model_name)
    scores = cross_validate(model, X_train_scale_ohe, y_train, cv=5, return_train_score=True)
    avg_scores[model_name] = pd.DataFrame(scores).drop(columns=['score_time']).mean()

In [147]:
pd.DataFrame(avg_scores).T

,fit_time,test_score,train_score
decision tree,0.415836,0.817568,0.999971
logistic regression,1.290177,0.851390,0.853645
RBF SVM,12.713557,0.637634,0.639338
random forest,3.931920,0.854922,0.999933


Let's discuss these results:

- Which methods are overfitting?
- Which methods are underfitting?
- Which methods are fast/slow?
- What is the best method so far?
- What hyperparameters should we tune?

In [148]:
models = {'decision tree'      : DecisionTreeClassifier(),
          'logistic regression orig': LogisticRegression(max_iter=300),
          'logistic regression': LogisticRegression(max_iter=300, C=100),
          'random forest orig'      : RandomForestClassifier(), 
          'random forest simpler'   : RandomForestClassifier(n_estimators=10)
         }
avg_scores = dict()

for model_name, model in models.items():
    print(model_name)
    scores = cross_validate(model, X_train_scale_ohe, y_train, cv=5, return_train_score=True)
    avg_scores[model_name] = pd.DataFrame(scores).drop(columns=['score_time']).mean()

decision tree
logistic regression orig
logistic regression
random forest orig
random forest simpler


In [149]:
pd.DataFrame(avg_scores).T

,fit_time,test_score,train_score
decision tree,0.475908,0.814919,0.999971
logistic regression orig,1.175866,0.851390,0.853645
logistic regression,1.364970,0.851313,0.853741
random forest orig,3.934673,0.854807,0.999914
random forest simpler,0.430480,0.847359,0.987245


## Encoding of categorical variables (25 mins)

From GH issue:

- first ask yourself "could you get new unseen values in deployment"
- if you make an "unknown" designation then in deployment, 3 different unknown categories would be treated the same. good to think about those types of nuances.

- In scikit-learn, most algorithms require numeric inputs.
- That means we have to convert categorical features to numeric values.
- We will do this twice: with a toy dataset for illustration, and then with a bigger dataset to compute scores.

- First dataset: we are looking at the `effect` of different treatment for depression.
- We would like to use `age`, `weight`, and `treatment` to predict `effect`.

In [90]:
df = pd.read_csv('data/depression_data.csv')
df.head()

,age,weight,treatment,effect
0,21,57.8,A,56
1,23,60.3,B,41
2,30,60.5,B,40
3,19,38.7,C,28
4,28,96.5,A,55


In [91]:
X = df[['age', 'weight', 'treatment']]
y = df['effect'] 

- Let's try to build a `LogisticRegression` classifier on this data?
- This will fail because we have non-numeric data.

In [92]:
lr = LogisticRegression()
lr.fit(X, y)

/Users/mgelbart/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'C'

#### No encoding (not recommended!)

- For starters, let's just delete the categorical feature.

In [93]:
X_drop = X.drop(columns=["treatment"])
X_drop.head()

,age,weight
0,21,57.8
1,23,60.3
2,30,60.5
3,19,38.7
4,28,96.5


#### Ordinal encoding (occasionally recommended)

- Here we simply assign an integer to each of our unique categorical labels
- We can use sklearn's [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html).

In [95]:
from sklearn.preprocessing import OrdinalEncoder

In [108]:
oe = OrdinalEncoder(dtype='int')
oe.fit(X[['treatment']])

X_ord = X.copy()
X_ord['treatment_encoded'] = oe.transform(X[['treatment']])
X_ord.head()

,age,weight,treatment,treatment_encoded
0,21,57.8,A,0
1,23,60.3,B,1
2,30,60.5,B,1
3,19,38.7,C,2
4,28,96.5,A,0


In [109]:
X_ord = X_ord.drop(columns='treatment')
X_ord.head()

,age,weight,treatment_encoded
0,21,57.8,0
1,23,60.3,1
2,30,60.5,1
3,19,38.7,2
4,28,96.5,0


- Can you see a big problem with this approach?
  - We have imposed ordinality on the categorical data.
  - For example, Treatment C (3) is "closer" to Treatment B (2) than Treatment A (1)
  - In general, label encoding is useful if there is ordinality in your data, e.g., `[cold, warm, hot]`.

#### One-hot encoding (OHE)
- Rather than assign integer labels to our data, we use create new binary columns to represent our categories.
- If we have $c$ categories in our column.
- We create $c$ new binary columns to represent those categories.
- We can use sklearn's [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

In [110]:
from sklearn.preprocessing import OneHotEncoder

In [111]:
ohe = OneHotEncoder(sparse=False, dtype='int')

In [112]:
ohe.fit(X[['treatment']])

X_ohe = pd.concat((X, pd.DataFrame(ohe.transform(X[['treatment']]),
                           columns=ohe.categories_[0], index=X.index)), axis=1)
X_ohe.head()

,age,weight,treatment,A,B,C
0,21,57.8,A,1,0,0
1,23,60.3,B,0,1,0
2,30,60.5,B,0,1,0
3,19,38.7,C,0,0,1
4,28,96.5,A,1,0,0


- We put a "1" for the actual value and "0" otherwise.
- But in fact, do we need all 3 columns?
  - After all, if `B` is 0 and `C` is 0, then `A` must be 1, right?
  - So, we drop one of the columns (usually the first one by convention).
- We can specify this in `OneHotEncoder()` by using the argument `drop='first'`

In [117]:
ohe = OneHotEncoder(drop='first', sparse=False, dtype='int')

ohe.fit(X[['treatment']])

X_ohe = pd.concat((X, pd.DataFrame(ohe.transform(X[['treatment']]),
                           columns=ohe.categories_[0][1:], index=X.index)), axis=1)
X_ohe.head()

,age,weight,treatment,B,C
0,21,57.8,A,0,0
1,23,60.3,B,1,0
2,30,60.5,B,1,0
3,19,38.7,C,0,1
4,28,96.5,A,0,0


In [118]:
X_ohe = X_ohe.drop(columns='treatment')
X_ohe.head()

,age,weight,B,C
0,21,57.8,0,0
1,23,60.3,1,0
2,30,60.5,1,0
3,19,38.7,0,1
4,28,96.5,0,0


- Can we do better? What about our discussion of scaling from earlier?
- Our features are not on the same scale and our encodings are getting "drowned out" by `age` and `weight`.
- We should preprocess both numeric features (e.g., scaling) and categorical features (e.g., OHE).
- sklearn's [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) makes this more manageable.
  - A big advantage here is that we build all our transformations together into one object, and that way we're sure we do the same operations to all splits of the data. 
  - Otherwise we might, for example, do the OHE on both train and test but forget to scale the test data.

In [119]:
from sklearn.compose import ColumnTransformer

In [120]:
# Identify the categorical and numeric columns
numeric_features = ['age', 'weight']
categorical_features = ['treatment']

In [122]:
transformers=[
    ('scale', StandardScaler(), numeric_features),
    ('ohe', OneHotEncoder(drop='first'), categorical_features)]

In [123]:
# Create the transformer
preprocessor = ColumnTransformer(transformers=transformers)

When we fit the preprocessor, it fits _all_ the transformers.

In [125]:
preprocessor.fit(X);

We can get the new names of the columns that were generated by the one-hot encoding:

In [126]:
preprocessor.named_transformers_['ohe'].get_feature_names(categorical_features)

array(['treatment_B', 'treatment_C'], dtype=object)

Combining this with the numeric feature names gives us all the column names:

In [127]:
columns = numeric_features + list(preprocessor.named_transformers_['ohe']
                                     .get_feature_names(categorical_features))
columns

['age', 'weight', 'treatment_B', 'treatment_C']

Like fit, when we transform with the preprocessor, it calls `transform` on _all_ the transformers.

In [129]:
# Apply data transformations and convert back to dataframe
X_ohe_scale = pd.DataFrame(preprocessor.transform(X),
                       index=X.index,
                       columns=columns)

In [130]:
X_ohe_scale.head()

,age,weight,treatment_B,treatment_C
0,-1.602301,-1.294360,0.0,0.0
1,-1.463640,-1.204307,1.0,0.0
2,-0.978328,-1.197103,1.0,0.0
3,-1.740961,-1.982364,0.0,1.0
4,-1.116989,0.099659,0.0,0.0


- Side note: the `ColumnTransformer` will automatically remove columns that are not being transformed:

In [132]:
preprocessor = ColumnTransformer(transformers=[
    ('ohe', OneHotEncoder(drop='first'), categorical_features)])

preprocessor.fit_transform(X)

array([[0., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.],
       [1., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 1.]])

Using `remainder='passthrough'` keeps the other columns in tact:

In [133]:
preprocessor = ColumnTransformer(transformers=[
    ('ohe', OneHotEncoder(drop='first'), categorical_features)], 
                                 remainder='passthrough')

preprocessor.fit_transform(X)

array([[  0. ,   0. ,  21. ,  57.8],
       [  1. ,   0. ,  23. ,  60.3],
       [  1. ,   0. ,  30. ,  60.5],
       [  0. ,   1. ,  19. ,  38.7],
       [  0. ,   0. ,  28. ,  96.5],
       [  0. ,   1. ,  23. ,  41.8],
       [  1. ,   0. ,  33. , 111. ],
       [  0. ,   1. ,  67. , 158. ],
       [  1. ,   0. ,  42. , 109.4],
       [  0. ,   0. ,  33. ,  66.5],
       [  0. ,   0. ,  33. , 111.4],
       [  0. ,   1. ,  56. , 114.7],
       [  0. ,   1. ,  45. ,  91.6],
       [  1. ,   0. ,  43. ,  89.8],
       [  0. ,   0. ,  38. , 102.4],
       [  0. ,   1. ,  37. ,  75.7],
       [  1. ,   0. ,  43. ,  87.4],
       [  0. ,   1. ,  27. ,  49.9],
       [  0. ,   0. ,  43. ,  94.2],
       [  1. ,   0. ,  45. , 132. ],
       [  1. ,   0. ,  48. , 113.5],
       [  0. ,   1. ,  47. , 136. ],
       [  0. ,   0. ,  48. ,  95.3],
       [  0. ,   0. ,  53. , 121.6],
       [  1. ,   0. ,  58. , 121.1],
       [  0. ,   1. ,  29. ,  89.2],
       [  0. ,   0. ,  53. , 107.3],
 

#### Categorical labels

- For now sklearn is fine with categorical labels ($y$-values), so we're not preprocessing them.


#### Side note: 

- Theoretically, decision trees should have no problem with categorical inputs
  - e.g., "is the treatment equal to C?"
  - However, the sklearn implementation does not support this, so we need to convert the features to numerical.
- On the other hand, logistic regression fundamentally operates on numerical data
  - So any implementation would require these transformations.

## Break (5 min)

## Encoding categorical variables, revisited (15 min)

In [92]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


Note: we'll ignore missing values for now, and return to them shortly.

In [93]:
# Let's identify numeric and categorical features

numeric_features = ['age', 'fnlwgt', 'education.num', 
                    'capital.gain', 'capital.loss', 
                    'hours.per.week']

categorical_features = ['workclass', 'education', 'marital.status', 
                        'occupation', 'relationship', 
                        'race', 'sex', 'native.country']

In [94]:
df.shape

(32561, 15)

Create `X` and `y` from the DataFrame:

In [95]:
X_train = df_train.drop(['income'], axis=1)
y_train = df_train['income']

X_test = df_test.drop(['income'], axis=1)
y_test = df_test['income']

- We will skip `ColumnTransformer` for now and just scale/encode separately for simplicity.
- Let's first scale the numeric features.

In [96]:
sc = StandardScaler()
sc.fit(X_train[numeric_features]);

In [97]:
X_train_scale_numeric = pd.DataFrame(sc.transform(X_train[numeric_features]),
                           columns=numeric_features, index=X_train.index)
X_test_scale_numeric = pd.DataFrame(sc.transform(X_test[numeric_features]),
                           columns=numeric_features, index=X_test.index)

X_train_scale_numeric.head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
17064,-1.364769,-0.745073,-0.027403,-0.146669,-0.216807,-0.845065
18434,-1.217915,0.708967,-0.416178,-0.146669,-0.216807,3.204805
3294,0.911476,1.397806,-0.416178,0.451354,-0.216807,1.179870
31317,1.058330,-0.736111,1.138922,-0.146669,-0.216807,-0.440078
4770,0.984903,-1.316034,1.138922,-0.146669,-0.216807,1.179870


Let's re-combine this with the categorical features:

In [98]:
X_train_scale = X_train.copy()
X_train_scale.update(X_train_scale_numeric)

X_test_scale = X_test.copy()
X_test_scale.update(X_test_scale_numeric)

X_train_scale.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
17064,-1.364769,Private,-0.745073,Some-college,-0.027403,Never-married,Adm-clerical,Own-child,Asian-Pac-Islander,Female,-0.146669,-0.216807,-0.845065,United-States
18434,-1.217915,Private,0.708967,HS-grad,-0.416178,Never-married,Other-service,Own-child,Black,Male,-0.146669,-0.216807,3.204805,United-States
3294,0.911476,Private,1.397806,HS-grad,-0.416178,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.451354,-0.216807,1.179870,United-States
31317,1.058330,Private,-0.736111,Bachelors,1.138922,Married-civ-spouse,Other-service,Husband,White,Male,-0.146669,-0.216807,-0.440078,United-States
4770,0.984903,Self-emp-inc,-1.316034,Bachelors,1.138922,Married-civ-spouse,Sales,Husband,White,Male,-0.146669,-0.216807,1.179870,United-States


#### Drop categorical features

In [99]:
X_train_scale_drop = X_train_scale.drop(columns=categorical_features)
X_test_scale_drop = X_test_scale.drop(columns=categorical_features)

X_train_scale_drop.head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
17064,-1.364769,-0.745073,-0.027403,-0.146669,-0.216807,-0.845065
18434,-1.217915,0.708967,-0.416178,-0.146669,-0.216807,3.204805
3294,0.911476,1.397806,-0.416178,0.451354,-0.216807,1.179870
31317,1.058330,-0.736111,1.138922,-0.146669,-0.216807,-0.440078
4770,0.984903,-1.316034,1.138922,-0.146669,-0.216807,1.179870


In [100]:
lr_drop = LogisticRegression()
lr_drop.fit(X_train_scale_drop, y_train);

In [101]:
show_scores(lr_drop, X_train_scale_drop, y_train, X_test_scale_drop, y_test)

Training error: 0.185
Test     error: 0.185


#### Ordinal encoding

In [102]:
oe = OrdinalEncoder()
oe.fit(X_train_scale[categorical_features]);

In [103]:
X_train_ord_cat = pd.DataFrame(oe.transform(X_train_scale[categorical_features]),
                           columns=categorical_features, index=X_train_scale.index)
X_test_ord_cat = pd.DataFrame(oe.transform(X_test_scale[categorical_features]),
                           columns=categorical_features, index=X_test_scale.index)
X_train_ord_cat.head()

,workclass,education,marital.status,occupation,relationship,race,sex,native.country
17064,4.0,15.0,4.0,1.0,3.0,1.0,0.0,39.0
18434,4.0,11.0,4.0,8.0,3.0,2.0,1.0,39.0
3294,4.0,11.0,2.0,4.0,0.0,4.0,1.0,39.0
31317,4.0,9.0,2.0,8.0,0.0,4.0,1.0,39.0
4770,5.0,9.0,2.0,12.0,0.0,4.0,1.0,39.0


In [104]:
X_train_scale_ord = X_train_scale.copy()
X_train_scale_ord.update(X_train_ord_cat)
X_test_scale_ord = X_test_scale.copy()
X_test_scale_ord.update(X_test_ord_cat)

X_train_scale_ord.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
17064,-1.364769,4,-0.745073,15,-0.027403,4,1,3,1,0,-0.146669,-0.216807,-0.845065,39
18434,-1.217915,4,0.708967,11,-0.416178,4,8,3,2,1,-0.146669,-0.216807,3.204805,39
3294,0.911476,4,1.397806,11,-0.416178,2,4,0,4,1,0.451354,-0.216807,1.179870,39
31317,1.058330,4,-0.736111,9,1.138922,2,8,0,4,1,-0.146669,-0.216807,-0.440078,39
4770,0.984903,5,-1.316034,9,1.138922,2,12,0,4,1,-0.146669,-0.216807,1.179870,39


In [105]:
lr_ord = LogisticRegression()
lr_ord.fit(X_train_scale_ord, y_train);

In [106]:
show_scores(lr_ord, X_train_scale_ord, y_train, X_test_scale_ord, y_test)

Training error: 0.176
Test     error: 0.174


Let's compare with the case where we dropped categorical variables:

In [107]:
show_scores(lr_drop, X_train_scale_drop, y_train, X_test_scale_drop, y_test)

Training error: 0.185
Test     error: 0.185


Ok, we're doing a bit better now.

- Returning to the problem with this approach:
  - We have imposed ordinality on the categorical data.
  - For example:

In [108]:
c = oe.categories_[3]
{i : c_i for i, c_i in enumerate(c)}

{0: '?',
 1: 'Adm-clerical',
 2: 'Armed-Forces',
 3: 'Craft-repair',
 4: 'Exec-managerial',
 5: 'Farming-fishing',
 6: 'Handlers-cleaners',
 7: 'Machine-op-inspct',
 8: 'Other-service',
 9: 'Priv-house-serv',
 10: 'Prof-specialty',
 11: 'Protective-serv',
 12: 'Sales',
 13: 'Tech-support',
 14: 'Transport-moving'}

- Is it reasonable to say "occupation > 3" in the above? There is no pattern to these occupations. 
- In this case, it may be reasonable for the education feature:

In [109]:
X_train["education"].unique()

array(['Some-college', 'HS-grad', 'Bachelors', 'Doctorate', '12th',
       '10th', 'Masters', '11th', 'Prof-school', 'Assoc-voc',
       'Assoc-acdm', '7th-8th', '9th', '1st-4th', 'Preschool', '5th-6th'],
      dtype=object)

- But we would want to set the order manually, not allow Python to do it arbitrarily (probably alphabetical)!
- Interestingly, this has already been done for us in this dataset:

In [110]:
X_train[['education', 'education.num']]

,education,education.num
17064,Some-college,10
18434,HS-grad,9
3294,HS-grad,9
31317,Bachelors,13
4770,Bachelors,13
...,...,...
28636,HS-grad,9
17730,10th,6
28030,Some-college,10
15725,5th-6th,3


So, in reality, we might just drop the `education` column and only use `education.num`. Here we'll just ignore this issue and keep both.

#### OHE

In [111]:
ohe = OneHotEncoder(drop='first', sparse=False, dtype=int)
ohe.fit(X_train_scale[categorical_features]);

In [112]:
X_train_scale_ohe = pd.concat((X_train_scale, pd.DataFrame(ohe.transform(X_train_scale[categorical_features]),
                           columns=ohe.get_feature_names(categorical_features), index=X_train_scale.index)), axis=1).drop(columns=categorical_features)
X_test_scale_ohe = pd.concat((X_test_scale, pd.DataFrame(ohe.transform(X_test_scale[categorical_features]),
                           columns=ohe.get_feature_names(categorical_features), index=X_test_scale.index)), axis=1).drop(columns=categorical_features)

X_train_scale_ohe.head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
17064,-1.364769,-0.745073,-0.027403,-0.146669,-0.216807,-0.845065,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
18434,-1.217915,0.708967,-0.416178,-0.146669,-0.216807,3.204805,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3294,0.911476,1.397806,-0.416178,0.451354,-0.216807,1.179870,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
31317,1.058330,-0.736111,1.138922,-0.146669,-0.216807,-0.440078,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4770,0.984903,-1.316034,1.138922,-0.146669,-0.216807,1.179870,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [113]:
X_train_scale_ohe.shape

(26048, 100)

In [114]:
lr_ohe = LogisticRegression()
lr_ohe.fit(X_train_scale_ohe, y_train);

In [115]:
show_scores(lr_ohe, X_train_scale_ohe, y_train, X_test_scale_ohe, y_test)

Training error: 0.147
Test     error: 0.149


Again, going back to our previous scores:

In [116]:
show_scores(lr_ord, X_train_scale_ord, y_train, X_test_scale_ord, y_test)

Training error: 0.176
Test     error: 0.174


In [117]:
show_scores(lr_drop, X_train_scale_drop, y_train, X_test_scale_drop, y_test)

Training error: 0.185
Test     error: 0.185


We see that we've indeed done better!